In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cta-ridership-data/CTA_-_Ridership_-_Daily_Boarding_Totals_20241108.csv
/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv


In [7]:
cta_by_st = pd.read_csv('/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv')

print(cta_by_st.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides'], dtype='object')


W=Weekday, A=Saturday, U=Sunday/Holiday

In [8]:
# Ensure 'date' is in datetime format
cta_by_st['date'] = pd.to_datetime(cta_by_st['date'])

# Create a 'year' column by extracting the year from the 'date' column
cta_by_st['year'] = cta_by_st['date'].dt.year

# Group by station and year, and sum the ridership ('rides')
cta_yearly = cta_by_st.groupby(['stationname', 'year'])['rides'].sum().reset_index()

# Display the first few rows of the new dataframe
print(cta_yearly.head())

  stationname  year   rides
0        18th  2001  256183
1        18th  2002  251387
2        18th  2003  251773
3        18th  2004  236536
4        18th  2005  320783


In [12]:
plt.figure(figsize=(12, 8))  # Adjust figure size
for station in stations:
    station_data = cta_yearly[cta_yearly['stationname'] == station]
    plt.plot(station_data['year'], station_data['rides'], label=station)

# Add labels and title
plt.title('Total Ridership by Year for Each Station')
plt.xlabel('Year')
plt.ylabel('Total Rides')
plt.legend(title='Station', bbox_to_anchor=(1.1, 1), loc='upper left')
plt.xticks(cta_yearly['year'].unique())
plt.tight_layout()

# Save the plot to a file
plt.savefig('ridership_by_year.png', bbox_inches='tight')  # Save with tight layout to avoid clipping

# Close the plot to free up memory
plt.close()

/tmp/ipykernel_31/1548493258.py:12: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  plt.tight_layout()


In [13]:
cta_st_info = pd.read_csv('/kaggle/input/cta-st-info/CTA_-_System_Information_-_List_of__L__Stops_20241108.csv')

print(cta_st_info.columns)

Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [17]:
print(cta_by_st.columns)
print(cta_st_info.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides', 'year'], dtype='object')
Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [18]:
# Rename 'stationname' in cta_by_st to 'STATION_NAME' to match cta_st_info
cta_by_st.rename(columns={'stationname': 'STATION_NAME'}, inplace=True)

# Merge the two DataFrames on 'STATION_NAME'
cta_stn_merged = pd.merge(cta_by_st, cta_st_info[['STATION_NAME', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']], 
                          on='STATION_NAME', how='left')

# Check the first few rows of the merged DataFrame
print(cta_stn_merged.head())

   station_id      STATION_NAME       date daytype  rides  year    RED   BLUE  \
0       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
1       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
2       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
3       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
4       40280      Central-Lake 2017-12-02       A   1270  2017    NaN    NaN   

       G    BRN      P   Pexp      Y    Pnk      O  
0  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  
3  False  False  False  False  False  False  False  
4    NaN    NaN    NaN    NaN    NaN    NaN    NaN  


In [19]:
# Export the merged DataFrame to a CSV file
cta_stn_merged.to_csv('cta_stn_merged.csv', index=False)

In [44]:
# Define the line columns to aggregate
line_columns = ['RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']

# Group by year and station and sum the 'rides' and the line columns
cta_ridership_total = cta_stn_merged.groupby(['year', 'STATION_NAME'], as_index=False)[['rides'] + line_columns].sum()

# If 'RED' and other line columns exist, apply the transformation
for line in line_columns:
    cta_ridership_total[line] = cta_ridership_total[line].apply(lambda x: 1 if x > 0 else 0)

# Check the result
cta_ridership_total


,year,STATION_NAME,rides,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
0,2001,18th,512366,0,0,0,0,0,0,0,1,0
1,2001,35-Bronzeville-IIT,567806,0,0,0,0,0,0,0,0,0
2,2001,35th/Archer,1254906,0,0,0,0,0,0,0,0,1
3,2001,43rd,495980,0,0,1,0,0,0,0,0,0
4,2001,47th-Dan Ryan,833311,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3424,2024,Western-Cermak,189322,0,0,0,0,0,0,0,0,0
3425,2024,Western-Forest Park,134776,0,0,0,0,0,0,0,0,0
3426,2024,Western-Orange,482034,0,0,0,0,0,0,0,0,0
3427,2024,Western/Milwaukee,653798,0,0,0,0,0,0,0,0,0


In [50]:
# Create 'active_lines' column by summing across the line columns
cta_ridership_total['active_lines'] = cta_ridership_total[line_columns].sum(axis=1)

# Now calculate the correlation between 'rides' and 'active_lines'
correlation = cta_ridership_total['rides'].corr(cta_ridership_total['active_lines'])
print(f"Correlation between ridership and number of active lines: {correlation}")


Correlation between ridership and number of active lines: 0.5622907298577524


The correlation of 0.562 between ridership and the number of active lines indicates a moderate positive relationship. This means that as the number of active lines at a station increases, the ridership tends to increase as well, but the correlation is not very strong. There may be other factors influencing ridership that are not captured by the number of active lines alone.

In [52]:
# Data for train line capacities
# Define the train lines and their respective capacities
train_lines = {
    'RED': {'typical_capacity': 8, 'max_length': 8},
    'BLUE': {'typical_capacity': 8, 'max_length': 8},
    'G': {'typical_capacity': 8, 'max_length': 8},
    'BRN': {'typical_capacity': 6, 'max_length': 6},
    'P': {'typical_capacity': 8, 'max_length': 8},
    'Pexp': {'typical_capacity': 8, 'max_length': 8},
    'Y': {'typical_capacity': 2, 'max_length': 2},
    'Pnk': {'typical_capacity': 8, 'max_length': 8},
    'O': {'typical_capacity': 8, 'max_length': 8}
}

# Convert this dictionary into a DataFrame for easier handling
line_capacity_df = pd.DataFrame.from_dict(train_lines, orient='index')
line_capacity_df.reset_index(inplace=True)
line_capacity_df.rename(columns={'index': 'Line'}, inplace=True)


line_capacity_df

,Line,typical_capacity,max_length
0,RED,8,8
1,BLUE,8,8
2,G,8,8
3,BRN,6,6
4,P,8,8
5,Pexp,8,8
6,Y,2,2
7,Pnk,8,8
8,O,8,8
